In [21]:
#install and import mlflow

In [1]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import load_iris
import mlflow
from mlflow.models.signature import infer_signature
import mlflow.sklearn
import pandas as pd

In [2]:
# Load dataset and split into training and test sets
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="target")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# Save dataset locally for logging in MLflow
X_train.to_csv("X_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
y_test.to_csv("y_test.csv", index=False)

In [5]:
# Set MLflow experiment
mlflow.set_experiment("RandomForest_Hyperparameter_Tuning")

<Experiment: artifact_location='file:///Users/sarath/Desktop/mlop/mlruns/138322625717650324', creation_time=1735463065825, experiment_id='138322625717650324', last_update_time=1735463065825, lifecycle_stage='active', name='RandomForest_Hyperparameter_Tuning', tags={}>

In [6]:
# Hyperparameter grid
param_grid = {
    "n_estimators": [50, 100, 150],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10]
}

In [7]:
# Initialize GridSearchCV
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring="accuracy")

In [8]:
#Start a parent MLflow run
with mlflow.start_run(run_name="Hyperparameter_Tuning") as parent_run:
    # Log datasets
    mlflow.log_artifact("X_train.csv", artifact_path="datasets")
    mlflow.log_artifact("y_train.csv", artifact_path="datasets")
    mlflow.log_artifact("X_test.csv", artifact_path="datasets")
    mlflow.log_artifact("y_test.csv", artifact_path="datasets")
    mlflow.set_tag("Dataset", "Iris Dataset")  # Set dataset tag for easy reference in the "Dataset" column

In [9]:
   # Train models with GridSearchCV
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]},
             scoring='accuracy')

In [10]:
 # Log each parameter set and corresponding accuracy in nested runs
for params, mean_score in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score']):
    with mlflow.start_run(run_name="GridSearchCV_Run", nested=True):
         mlflow.log_params(params)
         mlflow.log_metric("mean_accuracy", mean_score)

In [11]:
 # Log the best parameters and score at the parent level
best_params = grid_search.best_params_
best_score = grid_search.best_score_
mlflow.log_params(best_params)
mlflow.log_metric("best_cross_validated_accuracy", best_score)

In [13]:
# Evaluate on test data
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
mlflow.log_metric("test_accuracy", test_accuracy)

In [14]:
# Evaluate on test data to generate predictions
input_example = X_test.iloc[:1]  # Use one row from X_test as the input example
predictions = best_model.predict(X_test)
signature = infer_signature(X_test, predictions)

In [15]:
# Print and log the classification report
class_report = classification_report(y_test, y_pred, target_names=data.target_names)
print("\nClassification Report:\n", class_report)


Classification Report:
               precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [16]:
# Log the classification report as an artifact
with open("classification_report.txt", "w") as f:
    f.write(class_report)
mlflow.log_artifact("classification_report.txt")

In [17]:
# Log the best model with input_example and signature
mlflow.sklearn.log_model(
    sk_model=best_model,
    artifact_path="RandomForest_Best_Model",
    registered_model_name="RandomForestModel",
    input_example=input_example,
    signature=signature
)


Registered model 'RandomForestModel' already exists. Creating a new version of this model...
Created version '5' of model 'RandomForestModel'.
